In [223]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re, os, warnings
from astropy.io import fits
from astropy.wcs import WCS
import astropy.units as u
from astropy.utils.exceptions import AstropyWarning
import astropy.visualization as viz # MinMaxInterval, ImageNormalize, LinearStretch, ZScaleInterval, SqrtStretch, LogStretch
import astropy.coordinates as coords
import yaml
from hidefutils import id_to_name, mag_to_smass, normalize_image, jy_to_cm2, get_splus_phot

warnings.simplefilter('ignore', category=AstropyWarning)
warnings.filterwarnings(action='ignore', category=UserWarning, message='converting a masked element to nan ')

font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 16}
plt.rc('font', **font)

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
    "axes.linewidth": 1.5
})

In [225]:
def smass(hcgno):
    with open('data_files.yml') as f:
        grp_params = yaml.safe_load(f)
    dist = grp_params[f'HCG {hcgno}']['distance']

    df_output = pd.DataFrame(columns=['ID', 'Name', 'RA', 'Dec', 'gmag', 'e_gmag', 'rmag', 'e_rmag', 'radius_r', 'ba_ratio', 'radius_g', 'logMstar', 'e_logMstar'])
    id_dict = id_to_name(f'HCG {hcgno}')
    if str(hcgno) == '91':
        df_src = pd.read_csv('sources_h91.csv')
        ra, dec = df_src.ra, df_src.dec
        coo = coords.SkyCoord(ra, dec, unit=('deg', 'deg'), frame='fk5')
        df_splus  = pd.read_csv('phot_hcg91_splus.csv')
        sra, sdec = df_splus.RA, df_splus.DEC
        coo_splus = coords.SkyCoord(sra, sdec, unit=('deg', 'deg'), frame='fk5')
        idx, idxsplus, d2d, d3d = coords.search_around_sky(coo, coo_splus, 0.5*u.arcmin)
        id_uniq, id_cnt = np.unique(idx, return_counts=True)
        match_ids = np.zeros(id_uniq.size, dtype=int)
        for i, (id_, cnt) in enumerate(zip(id_uniq, id_cnt)):
            if cnt == 1:
                for j in range(len(idx)):
                    if idx[j] == id_:
                        match_ids[i] = idxsplus[j]
            else:
                m = idxsplus[idx==id_]
                d2d_ = d2d[idx==id_]
                mind2d = np.where(d2d_ == d2d_.min())
                match_ids[i] = m[mind2d[0][0]]
        sub_splus = df_splus.iloc[match_ids].reset_index(drop=True)
        sub_src = df_src.iloc[id_uniq].reset_index(drop=True)
        df_match = sub_src.join(sub_splus, rsuffix='_splus').drop(columns='sidelength(arcmin)').set_index('id')
        for i,galid in enumerate(df_match.index):
            galra, galdec = df_match.loc[galid].ra, df_match.loc[galid].dec
            rmag, gmag = df_match.loc[galid].r_auto, df_match.loc[galid].g_auto
            e_rmag, e_gmag = df_match.loc[galid].e_r_auto, df_match.loc[galid].e_g_auto
            radius = df_match.loc[galid].radius * 60
            ba_ratio = 1. / df_match.loc[galid].ab_ratio
            logms, e_logms = mag_to_smass(gmag=gmag, rmag=rmag, dist=dist, e_gmag=e_gmag, e_rmag=e_rmag)
            df_output.loc[i] = [galid, id_dict[galid], galra, galdec, gmag, e_gmag, rmag, e_rmag, radius, ba_ratio, np.nan, logms, e_logms]
        df_match.to_csv(f'photometry_match_hcg{hcgno}.csv', index=False)
    else:
        phot_r = pd.read_csv(f'desi_photometry/output/h{hcgno}_photometry_r.csv', index_col='ID')
        phot_g = pd.read_csv(f'desi_photometry/output/h{hcgno}_photometry_g.csv', index_col='ID')
        for i,galid in enumerate(id_dict.keys()):
            try:
                rmag, gmag = phot_r.loc[galid].petroMag, phot_g.loc[galid].petroMag
                radius_r, radius_g = phot_r['R25(arcsec)'].loc[galid]/60., phot_g['R25(arcsec)'].loc[galid]/60.
                ba_ratio = phot_r['b/a'].loc[galid]
            except KeyError:
                print('--- HCG %s: %s' %(hcgno, galid))
            else:
                galra, galdec = phot_r.loc[galid].RAobj, phot_r.loc[galid].DECobj
                e_rmag, e_gmag = phot_r.loc[galid].petroMag_err, phot_g.loc[galid].petroMag_err
                logms, e_logms = mag_to_smass(gmag=gmag, rmag=rmag, dist=dist, e_gmag=e_gmag, e_rmag=e_rmag)
                df_output.loc[i] = [galid, id_dict[galid], galra, galdec, gmag, e_gmag, rmag, e_rmag, radius_r, ba_ratio, radius_g, logms, e_logms]
    df_output.to_csv(f'stellar_masses_h{hcgno}.csv', index=False)
    return df_output

In [226]:
for hcg in [16, 30, 31, 90, 91, 97]:
    smass(hcg)

In [ ]:
# with open('group_members.yml') as f:
#     group_members = yaml.safe_load(f)
# with open('data_files.yml') as f:
#         params = yaml.safe_load(f)

# g = 'HCG 91'
# grz, opthdr = fits.getdata('./HCGs/legacy_images/%s_grz.fits' %g.replace(' ',''), header=True)
# grz_normalized = [normalize_image(image, contrast=1e-4, stretch_func='log', a=50) for image in grz]
# grz_normalized.reverse()

# df_splus  = pd.read_csv('phot_%s_splus.csv' %g.replace(' ','').lower())
# sra, sdec = df_splus.RA, df_splus.DEC

# rgb_image = np.stack(grz_normalized, axis=-1)
# levs = [6.6e18 * 2**x for x in range(5)]
# optwcs = WCS(opthdr).celestial

# fits_m0 = params[g]['rootdir'] + '/' + params[g]['moment_0']
# hidata, hihdr = fits.getdata(fits_m0, header=True)
# hiwcs = WCS(hihdr)
# nhi = jy_to_cm2(hidata,hihdr)

# fig = plt.figure(figsize=(8,8))
# ax = fig.add_subplot(111, projection=optwcs)
# if g == 'HCG 91':
#     image = grz[0]
#     ax.imshow(image, origin='lower', cmap='gray_r', norm=viz.ImageNormalize(image, interval=viz.PercentileInterval(99.), stretch=viz.SinhStretch(0.5)))
#     ax.contour(nhi, levels=levs, linewidths=0.5, colors='y', transform=ax.get_transform(hiwcs))
# else:
#     ax.imshow(rgb_image, origin='lower')
#     ax.contour(nhi, levels=levs, linewidths=0.5, colors='w', transform=ax.get_transform(hiwcs))
# ax.scatter(sra, sdec, c='r', marker='x', s=10, transform=ax.get_transform('world'))
# ax.set_xlim(0, rgb_image.shape[0])
# ax.set_ylim(0, rgb_image.shape[1])
# match_ra, match_dec = df_output.RA, df_output.Dec
# ax.scatter(match_ra, match_dec, ec='r', fc='None', marker='o', s=20, transform=ax.get_transform('world'))
# plt.savefig('splus_matches.pdf')
# plt.show()